# Conjuntos de Datos y DataLoaders

El código para procesar muestras de datos puede volverse desordenado y difícil de mantener; idealmente queremos que nuestro código de conjunto de datos
esté desacoplado de nuestro código de entrenamiento del modelo para mejor legibilidad y modularidad.
PyTorch proporciona dos primitivas de datos: ``torch.utils.data.DataLoader`` y ``torch.utils.data.Dataset``
que te permiten usar conjuntos de datos pre-cargados así como tus propios datos.
``Dataset`` almacena las muestras y sus etiquetas correspondientes, y ``DataLoader`` envuelve un iterable alrededor
del ``Dataset`` para habilitar acceso fácil a las muestras.

Las bibliotecas de dominio de PyTorch proporcionan un número de conjuntos de datos pre-cargados (como FashionMNIST) que
son subclases de ``torch.utils.data.Dataset`` e implementan funciones específicas para los datos particulares.
Pueden usarse para prototipar y hacer benchmark de tu modelo. Puedes encontrarlos
aquí: [Conjuntos de Datos de Imágenes](https://pytorch.org/vision/stable/datasets.html),
[Conjuntos de Datos de Texto](https://pytorch.org/text/stable/datasets.html), y
[Conjuntos de Datos de Audio](https://pytorch.org/audio/stable/datasets.html)

## Cargar un Conjunto de Datos

Aquí hay un ejemplo de cómo cargar el conjunto de datos [Fashion-MNIST](https://research.zalando.com/project/fashion_mnist/fashion_mnist/) de TorchVision.
Fashion-MNIST es un conjunto de datos de imágenes de artículos de Zalando que consiste en 60,000 ejemplos de entrenamiento y 10,000 ejemplos de prueba.
Cada ejemplo comprende una imagen en escala de grises de 28×28 y una etiqueta asociada de una de 10 clases.

Cargamos el [Conjunto de Datos FashionMNIST](https://pytorch.org/vision/stable/datasets.html#fashion-mnist) con los siguientes parámetros:
- ``root`` es la ruta donde se almacenan los datos de entrenamiento/prueba,
- ``train`` especifica conjunto de datos de entrenamiento o prueba,
- ``download=True`` descarga los datos de internet si no están disponibles en ``root``.
- ``transform`` y ``target_transform`` especifican las transformaciones de características y etiquetas

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

## Iterar y Visualizar el Conjunto de Datos

Podemos indexar ``Datasets`` manualmente como una lista: ``training_data[index]``.
Usamos ``matplotlib`` para visualizar algunas muestras en nuestros datos de entrenamiento.

In [ ]:
labels_map = {
    0: "Camiseta",
    1: "Pantalón", 
    2: "Suéter",
    3: "Vestido",
    4: "Abrigo",
    5: "Sandalia",
    6: "Camisa",
    7: "Zapatilla",
    8: "Bolso",
    9: "Botín",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Crear un Conjunto de Datos Personalizado para tus archivos

Una clase Dataset personalizada debe implementar tres funciones: `__init__`, `__len__`, y `__getitem__`.
Echa un vistazo a esta implementación; las imágenes de FashionMNIST están almacenadas
en un directorio ``img_dir``, y sus etiquetas están almacenadas separadamente en un archivo CSV ``annotations_file``.

En las siguientes secciones, desglosaremos qué está sucediendo en cada una de estas funciones.

In [ ]:
import os
import pandas as pd
from torchvision.io import decode_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = decode_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

### `__init__`

La función __init__ se ejecuta una vez al instanciar el objeto Dataset. Inicializamos
el directorio que contiene las imágenes, el archivo de anotaciones, y ambas transformaciones (cubiertas
en más detalle en la siguiente sección).

El archivo labels.csv se ve así:

```
tshirt1.jpg, 0
tshirt2.jpg, 0
......
ankleboot999.jpg, 9
```

In [ ]:
def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

### `__len__`

La función __len__ devuelve el número de muestras en nuestro conjunto de datos.

Ejemplo:

In [ ]:
def __len__(self):
    return len(self.img_labels)

### `__getitem__`

La función __getitem__ carga y devuelve una muestra del conjunto de datos en el índice dado ``idx``.
Basado en el índice, identifica la ubicación de la imagen en el disco, la convierte a un tensor usando ``read_image``, 
recupera la etiqueta correspondiente desde el archivo csv en ``self.img_labels``, llama a las funciones de transformación 
en ellos (si aplica), y devuelve la muestra del tensor y la etiqueta correspondiente en una tupla.

In [ ]:
def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = decode_image(img_path)
    label = self.img_labels.iloc[idx, 1]
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

---

## Preparar tus datos para el entrenamiento con DataLoaders

El ``Dataset`` recupera las características de nuestro conjunto de datos y etiqueta una muestra a la vez. Mientras entrenamos un modelo, típicamente queremos pasar muestras en "minibatches", reordenar los datos en cada época para reducir el overfitting del modelo, y usar el multiprocesamiento de Python para acelerar la recuperación de datos.

``DataLoader`` es un iterable que abstrae esta complejidad para nosotros en una API fácil.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

## Iterar a través del DataLoader

Hemos cargado ese conjunto de datos en el ``DataLoader`` y podemos iterar a través del conjunto de datos según sea necesario.
Cada iteración a continuación devuelve un lote de ``train_features`` y ``train_labels`` (conteniendo elementos de ``batch_size=64`` cada uno).
Debido a que especificamos ``shuffle=True``, después de que iteremos sobre todos los lotes, los datos se mezclan (para un control más detallado sobre el orden de carga de los datos, echa un vistazo a [Samplers](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler).

In [ ]:
# Mostrar imagen y etiqueta.
train_features, train_labels = next(iter(train_dataloader))
print(f"Forma del lote de características: {train_features.size()}")
print(f"Forma del lote de etiquetas: {train_labels.size()}")
print(train_features[0].shape)
img = train_features[0].squeeze()
print(img.shape)
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Etiqueta: {label}")

---

## Lectura Adicional
- [torch.utils.data API](https://pytorch.org/docs/stable/data.html)